In [1]:
import os
import cv2
import numpy as np
from tensorflow.keras.preprocessing.image import img_to_array

IMG_SIZE = (128, 128)

def preprocess_image(image_path):
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = cv2.resize(image, IMG_SIZE)
    image = img_to_array(image)
    image = np.expand_dims(image, axis=0)
    image = image / 255.0  # Normalize to [0, 1]
    return image


In [2]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

# Paths to the dataset directories
train_dir = 'chest_xray copy/train'
validation_dir = 'chest_xray copy/val'
test_dir = 'chest_xray copy/test'

# Image data generators
train_datagen = ImageDataGenerator(rescale=0.255, zoom_range=0.2, horizontal_flip=True)
validation_datagen = ImageDataGenerator(rescale=0.255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(128, 128),
    batch_size=32,
    class_mode='binary'
)

validation_generator = validation_datagen.flow_from_directory(
    validation_dir,
    target_size=(128, 128),
    batch_size=32,
    class_mode='binary'
)

# Build the CNN model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(
    train_generator,
    steps_per_epoch=100,
    epochs=10,
    validation_data=validation_generator,
    validation_steps=50
)

# Save the model
model.save('model/cnn_model.h5')


Found 5216 images belonging to 2 classes.
Found 16 images belonging to 2 classes.
Epoch 1/10
100/100 [==============================] - 38s 326ms/step - loss: 5.7883 - accuracy: 0.7866 - val_loss: 0.8314 - val_accuracy: 0.7500
Epoch 2/10
100/100 [==============================] - 30s 299ms/step - loss: 0.3397 - accuracy: 0.8709
Epoch 3/10
100/100 [==============================] - 29s 293ms/step - loss: 0.3171 - accuracy: 0.8844
Epoch 4/10
100/100 [==============================] - 33s 330ms/step - loss: 0.2946 - accuracy: 0.8806
Epoch 5/10
100/100 [==============================] - 33s 332ms/step - loss: 0.2667 - accuracy: 0.8941
Epoch 6/10
100/100 [==============================] - 31s 307ms/step - loss: 0.2651 - accuracy: 0.9003
Epoch 7/10
100/100 [==============================] - 28s 280ms/step - loss: 0.2729 - accuracy: 0.8863
Epoch 8/10
100/100 [==============================] - 28s 283ms/step - loss: 0.2466 - accuracy: 0.8959
Epoch 9/10
100/100 [==============================] 

In [8]:
import os
import cv2
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from preprocess import preprocess_image
import matplotlib.pyplot as plt

# Load the pre-trained model
model_path = 'model/cnn_model.h5'
model = load_model(model_path)

def predict_image(image_path):
    image = preprocess_image(image_path)
    prediction = model.predict(image)
    return prediction

def display_prediction(image_path, prediction):
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    plt.imshow(image)
    plt.title(f'Prediction: {"Pneumonia" if prediction > 0.5 else "Normal"}')
    plt.show()

# Directory paths
test_dir = 'chest_xray/test'

# Image data generator for test data
test_datagen = ImageDataGenerator(rescale=0.255)

# Load test data
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(128, 128),
    batch_size=32,
    class_mode='binary',
    shuffle=False
)

# Evaluate the model on the test data
test_loss, test_accuracy = model.evaluate(test_generator, steps=len(test_generator))
print(f'Test accuracy: {test_accuracy * 100:.2f}%')

# Optionally, display predictions for individual images
# image_dirs = ['NORMAL', 'PNEUMONIA']
# for image_dir in image_dirs:
#     full_dir = os.path.join(test_dir, image_dir)
#     for image_name in os.listdir(full_dir):
#         image_path = os.path.join(full_dir, image_name)
#         prediction = predict_image(image_path)
#         display_prediction(image_path, prediction)


Found 624 images belonging to 2 classes.
20/20 [==============================] - 5s 211ms/step - loss: 0.7052 - accuracy: 0.7949
Test accuracy: 79.49%


In [9]:
#Method 2

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.callbacks import ReduceLROnPlateau

# Paths to the dataset directories
train_dir = 'chest_xray/train'
validation_dir = 'chest_xray/val'
test_dir = 'chest_xray/test'

# Image data generators with enhanced augmentation
train_datagen = ImageDataGenerator(
    rescale=1.0/255.0,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

validation_datagen = ImageDataGenerator(rescale=1.0/255.0)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(128, 128),
    batch_size=32,
    class_mode='binary'
)

validation_generator = validation_datagen.flow_from_directory(
    validation_dir,
    target_size=(128, 128),
    batch_size=32,
    class_mode='binary'
)

# Build the enhanced CNN model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)),
    MaxPooling2D(pool_size=(2, 2)),
    BatchNormalization(),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    BatchNormalization(),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    BatchNormalization(),
    Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Learning rate reduction callback
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=0.00001)

# Train the model
model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    epochs=20,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size,
    callbacks=[reduce_lr]
)

# Save the model
model.save('model/cnn_model_enhanced.h5')

# Evaluate the model on the test data
test_datagen = ImageDataGenerator(rescale=1.0/255.0)
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(128, 128),
    batch_size=32,
    class_mode='binary',
    shuffle=False
)

test_loss, test_accuracy = model.evaluate(test_generator, steps=len(test_generator))
print(f'Test accuracy: {test_accuracy * 100:.2f}%')


Found 5216 images belonging to 2 classes.
Found 16 images belonging to 2 classes.
Epoch 1/20
163/163 [==============================] - 54s 327ms/step - loss: 0.8282 - accuracy: 0.8299 - lr: 0.0010
Epoch 2/20
163/163 [==============================] - 50s 307ms/step - loss: 0.3771 - accuracy: 0.8625 - lr: 0.0010
Epoch 3/20
163/163 [==============================] - 52s 319ms/step - loss: 0.3380 - accuracy: 0.8762 - lr: 0.0010
Epoch 4/20
163/163 [==============================] - 52s 315ms/step - loss: 0.3032 - accuracy: 0.8882 - lr: 0.0010
Epoch 5/20
163/163 [==============================] - 60s 371ms/step - loss: 0.2628 - accuracy: 0.8915 - lr: 0.0010
Epoch 6/20
163/163 [==============================] - 69s 423ms/step - loss: 0.2488 - accuracy: 0.8995 - lr: 0.0010
Epoch 7/20
163/163 [==============================] - 70s 427ms/step - loss: 0.2496 - accuracy: 0.9080 - lr: 0.0010
Epoch 8/20
163/163 [==============================] - 65s 398ms/step - loss: 0.2490 - accuracy: 0.9076 - l